# Abstract Matrices

In [1]:
import finite_algebras as alg
import numpy as np

import os
aa_path = os.path.join(os.getenv("PYPROJ"), "abstract_algebra")
alg_dir = os.path.join(aa_path, "Algebras")

# ex = alg.Examples(alg_dir)

In [2]:
ps3 = alg.generate_powerset_ring(3)
ps3.about()


** Ring **
Name: PSRing3
Instance ID: 4454732368
Description: Autogenerated Ring on powerset of {0, 1, 2} w/ symm. diff. (add) & intersection (mult)
Order: 8
Identity: {}
Commutative? Yes
Cyclic?: No
Elements:
   Index   Name   Inverse  Order
      0      {}      {}       1
      1     {0}     {0}       2
      2     {1}     {1}       2
      3     {2}     {2}       2
      4  {0, 1}  {0, 1}       2
      5  {0, 2}  {0, 2}       2
      6  {1, 2}  {1, 2}       2
      7 {0, 1, 2} {0, 1, 2}       2
Cayley Table (showing indices):
[[0, 1, 2, 3, 4, 5, 6, 7],
 [1, 0, 4, 5, 2, 3, 7, 6],
 [2, 4, 0, 6, 1, 7, 3, 5],
 [3, 5, 6, 0, 7, 1, 2, 4],
 [4, 2, 1, 7, 0, 6, 5, 3],
 [5, 3, 7, 1, 6, 0, 4, 2],
 [6, 7, 3, 2, 5, 4, 0, 1],
 [7, 6, 5, 4, 3, 2, 1, 0]]
Mult. Identity: {0, 1, 2}
Mult. Commutative? Yes
Zero Divisors: ['{0}', '{1}', '{2}', '{0, 1}', '{0, 2}', '{1, 2}']
Multiplicative Cayley Table (showing indices):
[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1],
 [0, 0, 2, 0, 2, 0, 2, 2],
 [0,

In [3]:
ps3.zero

'{}'

In [5]:
class AbstractMatrix:

    def __init__(self, array, ring):
        if isinstance(array, np.ndarray):
            arr = array
        else:
            arr = np.array(array)
        self.__ring = ring
        self.__array = np.array(array, dtype='<U32')
    
    @classmethod
    def zeros(cls, shape, ring):
        arr = np.full(shape, ring.zero, dtype='<U32')
        return cls(arr, ring)
    
    @classmethod
    def random(cls, shape, ring):
        rand_indices = np.random.randint(ring.order, size=shape)
        rand_array = np.full(shape, ring.zero, dtype='<U32')
        for i in range(shape[0]):
            for j in range(shape[1]):
                rand_array[i, j] = ring.elements[rand_indices[i, j]]
        return cls(rand_array, ring)
    
    def get_array(self):
        return self.__array
    
    def get_shape(self):
        return self.__array.shape
    
    def get_nrows(self):
        return self.__array.shape[0]
    
    def get_ncols(self):
        return self.__array.shape[1]
    
    def get_algebra(self):
        return self.__ring
    
    def copy(self):
        return AbstractMatrix(np.copy(self.__array), self.__ring)
    
    def transpose(self):
        return AbstractMatrix(np.transpose(self.__array), self.__ring)
    
    def __mul__(self, other):  # Matrix multiplication using Ring operations
        # X * Y
        xarr =  self.__array
        yarr = other.__array
        xrows = self.get_nrows()
        xcols = self.get_ncols()
        yrows = other.get_nrows()
        ycols = other.get_ncols()
        product = None
        if xcols == yrows:
            if self.__ring == other.__ring:
                ring = self.__ring
                product = np.full((xrows, ycols), ring.zero, dtype='U32')
                for i in range(xrows):
                    for j in range(ycols):
                        for k in range(xcols):
                            product[i, j] = ring.add(product[i, j], ring.mult(xarr[i, k], yarr[k, j]))
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are incompatible: {xcols} colums vs {yrows} rows")
        return AbstractMatrix(product, ring)
    
    def __add__(self, other):  # Matrix addition using Ring operations
        # X + Y
        xarr =  self.__array
        yarr = other.__array
        xshape = xarr.shape
        yshape = yarr.shape
        sum = None
        if xshape == yshape:
            if self.__ring == other.__ring:
                ring = self.__ring
                sum = np.full(xshape, ring.zero, dtype='U32')
                for i in range(xshape[0]):
                    for j in range(xshape[1]):
                        sum[i, j] = ring.add(xarr[i, j], yarr[i, j])
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are not equal: {xshape} != {yshape}")
        return AbstractMatrix(sum, ring)
    
    def __sub__(self, other):  # Matrix subtraction using Ring operations
        # X - Y
        xarr =  self.__array
        yarr = other.__array
        xshape = xarr.shape
        yshape = yarr.shape
        sum = None
        if xshape == yshape:
            if self.__ring == other.__ring:
                ring = self.__ring
                sum = np.full(xshape, ring.zero, dtype='U32')
                for i in range(xshape[0]):
                    for j in range(xshape[1]):
                        sum[i, j] = ring.sub(xarr[i, j], yarr[i, j])
            else:
                raise ValueError("The array algebras must be equal")
        else:
            raise ValueError(f"The array shapes are not equal: {xshape} != {yshape}")
        return AbstractMatrix(sum, ring)
    
    def determinant(self):
        return array_determinant(self.__array, self.__ring)
    
    def cofactor_matrix(self):
        return AbstractMatrix(array_cofactor(self.__array, self.__ring), self.__ring)
    
    def scalar_mult(self, scalar, left=True):
        '''Multiply self by a scalar. Default is left multiplication: scalar * self,
        otherwise right multiplication is used, self * scalar.'''
        if not scalar in self.__ring.elements:
            raise ValueError(f"{scalar} is not one of the matrix algebra elements")
        product = self.copy()
        for i in range(self.get_nrows()):
            for j in range(self.get_ncols()):
                if left:
                    product[i, j] = self.__ring.mult(scalar, product[i, j])
                else:
                    product[i, j] = self.__ring.mult(product[i, j], scalar)
        return product        

def array_determinant(array, ring):
    '''Compute the determinant of a square NumPy array of ring elements.'''
    nrows = array.shape[0]
    ncols = array.shape[1]
    if nrows != ncols:
        raise ValueError(f"Array must be square: ({nrows}, {ncols})")
    elif nrows == 1:
        return array[0, 0]
    elif nrows == 2:  # Recursion will stop here
        return ring.sub(ring.mult(array[0, 0], array[1, 1]), ring.mult(array[0, 1], array[1, 0]))
    else:
        # Use the Laplace expansion to recursively compute the determinant
        det = ring.zero  # The ring's "zero" element
        arr = np.delete(array, 0, 0)  # Copy array & delete first row
        for i in range(ncols):
            minor = np.delete(arr, i, 1)  # Copy arr & delete the ith column
            # Alternate "adding" and "subtracting", per the Laplace expansion
            if (-1)**i == 1:
                det = ring.add(det, ring.mult(array[0, i], array_determinant(minor, ring)))
            else:
                det = ring.sub(det, ring.mult(array[0, i], array_determinant(minor, ring)))
        return det
    
def array_cofactor(array, ring):
    nrows = array.shape[0]
    ncols = array.shape[1]
    cof = AbstractMatrix.zeros(array.shape, ring).get_array()
    for i in range(nrows):
        for j in range(ncols):
            arr1 = np.delete(array, i, 0)
            arr2 = np.delete(arr1,  j, 1)
            if (-1)**(i + j) == 1:
                cof[i, j] = array_determinant(arr2, ring)
            else:
                cof[i, j] = ring.inv(array_determinant(arr2, ring))
    return cof

In [6]:
mat1 = AbstractMatrix.zeros((2, 3), ps3)

arr = mat1.get_array()
arr

array([['{}', '{}', '{}'],
       ['{}', '{}', '{}']], dtype='<U32')

In [7]:
fubar = [['{0}', '{1}', '{2}'],
         ['{2}', '{1}', '{0}']]

mat2 = AbstractMatrix(fubar, ps3)

mat2.get_array()

array([['{0}', '{1}', '{2}'],
       ['{2}', '{1}', '{0}']], dtype='<U32')

In [8]:
mat3 = mat2.transpose()
mat3.get_array()

array([['{0}', '{2}'],
       ['{1}', '{1}'],
       ['{2}', '{0}']], dtype='<U32')

In [9]:
mat2x3 = mat2 * mat3
mat2x3.get_array()

array([['{0, 1, 2}', '{1}'],
       ['{1}', '{0, 1, 2}']], dtype='<U32')

In [10]:
mat3x2 = mat3 * mat2
mat3x2.get_array()

array([['{0, 2}', '{}', '{}'],
       ['{}', '{}', '{}'],
       ['{}', '{}', '{0, 2}']], dtype='<U32')

In [11]:
try:
    mat2x2 = mat2 * mat2
    print(mat2x2.get_array())
except Exception as exc:
    print(exc)

The array shapes are incompatible: 3 colums vs 2 rows


In [12]:
mat2p2 = mat2 + mat2

mat2p2.get_array()

array([['{}', '{}', '{}'],
       ['{}', '{}', '{}']], dtype='<U32')

In [13]:
try:
    mat2p3 = mat2 + mat3
    print(mat2p3.get_array())
except Exception as exc:
    print(exc)

The array shapes are not equal: (2, 3) != (3, 2)


In [14]:
mat2m2 = mat2 - mat2
mat2.get_array()

array([['{0}', '{1}', '{2}'],
       ['{2}', '{1}', '{0}']], dtype='<U32')

## Numeric Determinant

Compute the determinant of a square numeric NumPy array:

In [15]:
def numeric_determinant(array):
    '''Compute the determinant of a square NumPy array.'''
    nrows = array.shape[0]
    ncols = array.shape[1]
    if nrows != ncols:
        raise ValueError(f"Array must be square: ({nrows}, {ncols})")
    elif nrows == 1:
        return array[0, 0]
    elif nrows == 2:  # Recursion will stop here
        return (array[0, 0] * array[1, 1]) - (array[0, 1] * array[1, 0])
    else:
        # Use the Laplace expansion to recursively compute the determinant
        det = 0
        arr = np.delete(array, 0, 0)  # Copy array & delete first row
        for i in range(ncols):
            minor = np.delete(arr, i, 1)  # Copy arr & delete the ith column
            det += (-1)**i * array[0, i] * numeric_determinant(minor)
        return det
    
def numeric_cofactor(array):
    nrows = array.shape[0]
    ncols = array.shape[1]
    cof = np.zeros(array.shape)
    for i in range(nrows):
        for j in range(ncols):
            arr1 = np.delete(array, i, 0)
            arr2 = np.delete(arr1,  j, 1)
            cof[i, j] = (-1)**(i + j) * numeric_determinant(arr2)
    return cof

In [16]:
test1 = np.array([[1, 2, 1],
                  [0, 3, 4],
                  [3, 1, 4]])

numeric_determinant(test1)

23

In [17]:
np.linalg.det(test1)

23.0

In [18]:
numeric_determinant(np.array([[7]]))

7

In [19]:
test2 = np.array([[1, 2, 1, 3],
                  [0, 3, 4, 1],
                  [3, 1, 4, 2],
                  [1, 2, 4, 3]])

numeric_determinant(test2)

48

In [20]:
np.linalg.det(test2)

48.00000000000001

In [21]:
numeric_cofactor(test1)

array([[ 8., 12., -9.],
       [-7.,  1.,  5.],
       [ 5., -4.,  3.]])

In [22]:
numeric_cofactor(test2)

array([[ 12.,  20., -16.,   4.],
       [  3.,  21.,   0., -15.],
       [ 21.,   3.,   0.,  -9.],
       [-27., -29.,  16.,  23.]])

## Abstract Determinant, Cofactor

### 2x2 Test

In [23]:
test2 = mat2x3.get_array()
test2

array([['{0, 1, 2}', '{1}'],
       ['{1}', '{0, 1, 2}']], dtype='<U32')

In [24]:
array_determinant(test2, mat2x3.get_algebra())

'{0, 2}'

In [25]:
mat2x3.determinant()

'{0, 2}'

In [26]:
ps3.mult('{0, 1, 2}', '{0, 1, 2}')

'{0, 1, 2}'

In [27]:
ps3.mult('{1}', '{1}')

'{1}'

In [28]:
ps3.sub('{0, 1, 2}', '{1}')

'{0, 2}'

In [29]:
array_cofactor(test2, ps3)

array([['{0, 1, 2}', '{1}'],
       ['{1}', '{0, 1, 2}']], dtype='<U32')

### 3x3 Test

In [30]:
# rnd1 = AbstractMatrix.random((3,3),ps3)
# rnd1.get_array()

In [31]:
rnd1 = np.array([['{1, 2}', '{0, 1, 2}', '{0, 2}'],
                 ['{0, 2}', '{}', '{1}'],
                 ['{0}', '{1}', '{0, 1}']], dtype='<U32')

In [32]:
array_determinant(rnd1, ps3)

'{0, 1}'

In [33]:
rnd1_mat = AbstractMatrix(rnd1, ps3)
print(rnd1_mat.get_array())

[['{1, 2}' '{0, 1, 2}' '{0, 2}']
 ['{0, 2}' '{}' '{1}']
 ['{0}' '{1}' '{0, 1}']]


In [34]:
rnd1_mat.determinant()

'{0, 1}'

In [35]:
array_cofactor(rnd1, ps3)

array([['{1}', '{0}', '{}'],
       ['{0, 1}', '{0, 1}', '{0, 1}'],
       ['{1}', '{0, 1, 2}', '{0, 2}']], dtype='<U32')

In [36]:
rnd1_mat_cof = rnd1_mat.cofactor_matrix()
rnd1_mat_cof.get_array()

array([['{1}', '{0}', '{}'],
       ['{0, 1}', '{0, 1}', '{0, 1}'],
       ['{1}', '{0, 1, 2}', '{0, 2}']], dtype='<U32')

### 4x4 Test

In [37]:
# rnd2 = AbstractMatrix.random((4,4),ps3)
# rnd2.get_array()

In [38]:
rnd2 = np.array([['{}', '{0, 1, 2}', '{0, 1, 2}', '{}'],
                 ['{0, 1}', '{0, 2}', '{1, 2}', '{2}'],
                 ['{0, 2}', '{}', '{}', '{0, 1}'],
                 ['{1}', '{0}', '{0, 2}', '{}']], dtype='<U32')

In [39]:
array_determinant(rnd2, ps3)

'{1, 2}'

In [40]:
array_cofactor(rnd2, ps3)

array([['{0}', '{0, 1, 2}', '{0}', '{0, 2}'],
       ['{}', '{1}', '{1}', '{2}'],
       ['{2}', '{}', '{}', '{1}'],
       ['{0, 1}', '{0, 1, 2}', '{0, 1, 2}', '{0}']], dtype='<U32')

In [41]:
rnd2_mat = AbstractMatrix(rnd2, ps3)

In [42]:
rnd2_mat.determinant()

'{1, 2}'

In [43]:
rnd2_mat.cofactor_matrix().get_array()

array([['{0}', '{0, 1, 2}', '{0}', '{0, 2}'],
       ['{}', '{1}', '{1}', '{2}'],
       ['{2}', '{}', '{}', '{1}'],
       ['{0, 1}', '{0, 1, 2}', '{0, 1, 2}', '{0}']], dtype='<U32')